# YOUR FIRST LAB
### Please read this section. This is valuable to get you prepared, even if it's a long read -- it's important stuff.

## Your first Frontier LLM Project

Let's build a useful LLM solution - in a matter of minutes.

By the end of this course, you will have built an autonomous Agentic AI solution with 7 agents that collaborate to solve a business problem. All in good time! We will start with something smaller...

Our goal is to code a new kind of Web Browser. Give it a URL, and it will respond with a summary. The Reader's Digest of the internet!!

Before starting, you should have completed the setup for [PC](../SETUP-PC.md) or [Mac](../SETUP-mac.md) and you hopefully launched this jupyter lab from within the project root directory, with your environment activated.

## If you're new to Jupyter Lab

Welcome to the wonderful world of Data Science experimentation! Once you've used Jupyter Lab, you'll wonder how you ever lived without it. Simply click in each "cell" with code in it, such as the cell immediately below this text, and hit Shift+Return to execute that cell. As you wish, you can add a cell with the + button in the toolbar, and print values of variables, or try out variations.  

I've written a notebook called [Guide to Jupyter](Guide%20to%20Jupyter.ipynb) to help you get more familiar with Jupyter Labs, including adding Markdown comments, using `!` to run shell commands, and `tqdm` to show progress.

## If you're new to the Command Line

Please see these excellent guides: [Command line on PC](https://chatgpt.com/share/67b0acea-ba38-8012-9c34-7a2541052665) and [Command line on Mac](https://chatgpt.com/canvas/shared/67b0b10c93a081918210723867525d2b).  

## If you'd prefer to work in IDEs

If you're more comfortable in IDEs like VSCode, Cursor or PyCharm, they both work great with these lab notebooks too.  
If you'd prefer to work in VSCode, [here](https://chatgpt.com/share/676f2e19-c228-8012-9911-6ca42f8ed766) are instructions from an AI friend on how to configure it for the course.

## If you'd like to brush up your Python

I've added a notebook called [Intermediate Python](Intermediate%20Python.ipynb) to get you up to speed. But you should give it a miss if you already have a good idea what this code does:    
`yield from {book.get("author") for book in books if book.get("author")}`

## I am here to help

If you have any problems at all, please do reach out.  
I'm available through the platform, or at ed@edwarddonner.com, or at https://www.linkedin.com/in/eddonner/ if you'd like to connect (and I love connecting!)  
And this is new to me, but I'm also trying out X/Twitter at [@edwarddonner](https://x.com/edwarddonner) - if you're on X, please show me how it's done 😂  

## More troubleshooting

Please see the [troubleshooting](troubleshooting.ipynb) notebook in this folder to diagnose and fix common problems. At the very end of it is a diagnostics script with some useful debug info.

## For foundational technical knowledge (eg Git, APIs, debugging) 

If you're relatively new to programming -- I've got your back! While it's ideal to have some programming experience for this course, there's only one mandatory prerequisite: plenty of patience. 😁 I've put together a set of self-study guides that cover Git and GitHub, APIs and endpoints, beginner python and more.

This covers Git and GitHub; what they are, the difference, and how to use them:  
https://github.com/ed-donner/agents/blob/main/guides/03_git_and_github.ipynb

This covers technical foundations:  
ChatGPT vs API; taking screenshots; Environment Variables; Networking basics; APIs and endpoints:  
https://github.com/ed-donner/agents/blob/main/guides/04_technical_foundations.ipynb

This covers Python for beginners, and making sure that a `NameError` never trips you up:  
https://github.com/ed-donner/agents/blob/main/guides/06_python_foundations.ipynb

This covers the essential techniques for figuring out errors:  
https://github.com/ed-donner/agents/blob/main/guides/08_debugging.ipynb

And you'll find other useful guides in the same folder in GitHub. Some information applies to my other Udemy course (eg Async Python) but most of it is very relevant for LLM engineering.

## If this is old hat!

If you're already comfortable with today's material, please hang in there; you can move swiftly through the first few labs - we will get much more in depth as the weeks progress. Ultimately we will fine-tune our own LLM to compete with OpenAI!

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Please read - important note</h2>
            <span style="color:#900;">The way I collaborate with you may be different to other courses you've taken. I prefer not to type code while you watch. Rather, I execute Jupyter Labs, like this, and give you an intuition for what's going on. My suggestion is that you carefully execute this yourself, <b>after</b> watching the lecture. Add print statements to understand what's going on, and then come up with your own variations. If you have a Github account, use this to showcase your variations. Not only is this essential practice, but it demonstrates your skills to others, including perhaps future clients or employers...</span>
        </td>
    </tr>
</table>
<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">This code is a live resource - keep an eye out for my emails</h2>
            <span style="color:#f71;">I push updates to the code regularly. As people ask questions, I add more examples or improved commentary. As a result, you'll notice that the code below isn't identical to the videos. Everything from the videos is here; but I've also added better explanations and new models like DeepSeek. Consider this like an interactive book.<br/><br/>
                I try to send emails regularly with important updates related to the course. You can find this in the 'Announcements' section of Udemy in the left sidebar. You can also choose to receive my emails via your Notification Settings in Udemy. I'm respectful of your inbox and always try to add value with my emails!
            </span>
        </td>
    </tr>
</table>
<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business value of these exercises</h2>
            <span style="color:#181;">A final thought. While I've designed these notebooks to be educational, I've also tried to make them enjoyable. We'll do fun things like have LLMs tell jokes and argue with each other. But fundamentally, my goal is to teach skills you can apply in business. I'll explain business implications as we go, and it's worth keeping this in mind: as you build experience with models and techniques, think of ways you could put this into action at work today. Please do contact me if you'd like to discuss more or if you have ideas to bounce off me.</span>
        </td>
    </tr>
</table>

In [2]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI (or Ollama)

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.  

If you'd like to use free Ollama instead, please see the README section "Free Alternative to Paid APIs", and if you're not sure how to do this, there's a full solution in the solutions folder (day1_with_ollama.ipynb).

## Troubleshooting if you have problems:

Head over to the [troubleshooting](troubleshooting.ipynb) notebook in this folder for step by step code to identify the root cause and fix it!

If you make a change, try restarting the "Kernel" (the python process sitting behind this notebook) by Kernel menu >> Restart Kernel and Clear Outputs of All Cells. Then try this notebook again, starting at the top.

Or, contact me! Message me or email ed@edwarddonner.com and we will get this to work.

Any concerns about API costs? See my notes in the README - costs should be minimal, and you can control it at every point. You can also use Ollama as a free alternative, which we discuss during Day 2.

In [3]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [4]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions

# Let's make a quick call to a Frontier model to get started, as a preview!

In [5]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! Welcome! I'm glad to have you here. How can I assist you today?


## OK onwards with our first project

In [6]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [7]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [8]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [9]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [10]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acqui

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```python
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]
```
To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

In [11]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [12]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

Oh, let me dust off my math skills for this one. 2 + 2 equals 4. Shocking, I know!


## And now let's build useful messages for GPT-4o-mini, using a function

In [13]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [14]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a

## Time to bring it together - the API for OpenAI is very simple!

In [15]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [16]:
summarize("https://edwarddonner.com")

'# Summary of Edward Donner\'s Website\n\nEdward Donner\'s website is a personal platform where he shares his interests and expertise in coding, LLMs (Large Language Models), and electronic music. He is the co-founder and CTO of Nebula.io, a company focused on utilizing AI to help individuals discover their potential through talent management solutions. Previously, he founded and led untapt, an AI startup acquired in 2021.\n\n## Announcements\n- **May 28, 2025**: Launch of a course titled "Connecting my courses – become an LLM expert and leader."\n- **May 18, 2025**: Announcement for the "2025 AI Executive Briefing."\n- **April 21, 2025**: Introduction of "The Complete Agentic AI Engineering Course."\n- **January 23, 2025**: Offering of an "LLM Workshop – Hands-on with Agents – resources."\n\nEdward invites visitors to connect with him for further engagement.'

In [17]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [18]:
display_summary("https://edwarddonner.com")

# Summary of Edward Donner's Website

Edward Donner's website serves as a personal and professional hub for his interests in coding, LLMs (Large Language Models), and electronic music. He is the co-founder and CTO of Nebula.io, a company focused on leveraging AI to enhance talent discovery and management. Previously, he founded an AI startup called untapt, which was acquired in 2021.

## Key Features:
- **Connect Four**: A game or platform where LLMs engage in tactics of diplomacy.
- **Upcoming Courses & Workshops**:
  - **Connecting my courses – Become an LLM expert and leader** (May 28, 2025)
  - **2025 AI Executive Briefing** (May 18, 2025)
  - **The Complete Agentic AI Engineering Course** (April 21, 2025)
  - **LLM Workshop – Hands-on with Agents** (January 23, 2025)

Edward encourages visitors to connect with him through various platforms and subscribe to his newsletter for updates.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [19]:
display_summary("https://cnn.com")

# Summary of CNN

CNN's website provides up-to-date news and video content across a variety of topics, including US and world news, politics, business, health, entertainment, and sports. Key features include live updates on significant global conflicts such as the Israel-Iran War, in which recent events detail missile strikes, military actions, and political implications involving US involvement.

## Recent Highlights:
- **Israel-Iran Conflict**: Escalating military actions have resulted in fears for civilians in both countries. Israeli and Iranian forces have engaged in strikes, with significant implications for regional stability. US President Trump is considering military options and has allowed time for diplomatic discussions.
- **Stranded Pilgrims**: Over 10,000 pilgrims are reportedly stuck in Saudi Arabia amid the growing tensions.
- **US Airline Operations**: At least two US airlines have paused flights to the Middle East due to the ongoing conflict.

Other sections of the site cover various topics like health tips, scientific advancements, and cultural articles, as well as a dedicated space for opinion pieces and analyses.

In [20]:
display_summary("https://anthropic.com")

# Anthropic Website Summary

Anthropic is an AI research and product company dedicated to developing AI technologies that prioritize safety and human benefit. Their primary offering is **Claude**, a suite of intelligent AI models aimed at enhancing productivity and collaboration.

## Key Features:
- **Claude Models**: 
  - **Claude Opus 4**: The latest and most advanced AI model introduced by Anthropic, designed for extensive coding and abilities to manage prolonged tasks.
  - **Claude Sonnet 4 and Claude Haiku 3.5**: Additional models with unique capabilities.

- **API Services**: Developers can create AI-powered applications using the Claude API, with comprehensive documentation available.

- **Research Commitment**: Anthropic emphasizes responsible AI development through ongoing research, transparency, and a commitment to societal impacts.

- **Education and Resources**: The *Anthropic Academy* offers learning resources and courses on developing with Claude.

## Recent Announcements:
- The latest models, Claude Opus 4 and Sonnet 4, were announced as part of their initiative to push the boundaries of AI capabilities.
- **ISO 42001 certification** has been achieved, showcasing their commitment to industry standards and practices.

## Core Values:
Anthropic focuses on intentional AI development practices, encompassing ethical considerations, safety protocols, and the long-term well-being of society, making bold advancements while also evaluating potential impacts.

For more information, users can explore their various plans, research details, and customer success stories through the website.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise, you experienced calling the Cloud API of a Frontier Model (a leading model at the frontier of AI) for the first time. We will be using APIs like OpenAI at many stages in the course, in addition to building our own LLMs.

More specifically, we've applied this to Summarization - a classic Gen AI use case to make a summary. This can be applied to any business vertical - summarizing the news, summarizing financial performance, summarizing a resume in a cover letter - the applications are limitless. Consider how you could apply Summarization in your business, and try prototyping a solution.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue - now try yourself</h2>
            <span style="color:#900;">Use the cell below to make your own simple commercial example. Stick with the summarization use case for now. Here's an idea: write something that will take the contents of an email, and will suggest an appropriate short subject line for the email. That's the kind of feature that might be built into a commercial email tool.</span>
        </td>
    </tr>
</table>

In [ ]:
# Step 1: Create your prompts

system_prompt = """
You are an e-mail wiriting assistant. You will summarize the contents of an e-mail into an appropriate short subject line for that e-mail.
Use the language the e-mail is written in."""
user_prompt = """
    Καλημέρα σας,
Σας αποστέλλουμε το API Key για την πρόσβαση στα Ανοιχτά Δεδομένα του ΓΕΜΗ:
 API Key: UqnN3dsYAm4aeH8r6apEom2Sah8dHMMU
Χρησιμοποιήστε το Swagger για δοκιμές και εξερεύνηση του API:
Swagger Docs
Αναλυτική τεχνική τεκμηρίωση:
Τεχνική Τεκμηρίωση API
Rate Limits :
Το API υποστηρίζει μέγιστο όριο 8 αιτημάτων ανά λεπτό (8 req/min).
Εάν χρειάζεστε υψηλότερο όριο, παρακαλούμε να μας στείλετε αίτημα στο support@uhc.gr, περιγράφοντας:
Τον λόγο χρήσης του API
Τον αναμενόμενο όγκο αιτημάτων
Το χρονικό διάστημα χρήσης
Response Format: Οι απαντήσεις επιστρέφονται σε μορφή JSON. Βεβαιωθείτε ότι η εφαρμογή σας διαχειρίζεται σωστά τα δεδομένα.
Error Handling:
Σφάλμα 429 Too Many Requests → Σημαίνει ότι ξεπεράσατε το όριο κλήσεων. Περιμένετε και δοκιμάστε ξανά.
Σφάλμα 401 Unauthorized → Ελέγξτε αν χρησιμοποιείτε σωστά το API Key.
Σφάλμα 500+ → Προσωρινό πρόβλημα στον server, δοκιμάστε ξανά αργότερα.
Για πλήρη αναφορά, χρησιμοποιήστε τα Swagger Docs ή δείτε την τεκμηρίωση του API.
Τεχνική Υποστήριξη: Η Υπηρεσία Υποστήριξης και Ανάπτυξης των ΠΣ ΓΕΜΗ και ΥΜΣ είναι αρμόδια μόνο για τεχνικά θέματα σχετικά με την ακεραιότητα των συστημάτων και της βάσης δεδομένων.
Νομικά & Διαδικαστικά Ερωτήματα: Για θέματα που αφορούν τη χρήση ή ερμηνεία των δεδομένων, απευθυνθείτε στα Εμπορικά Επιμελητήρια και τη Γενική Γραμματεία Εμπορίου.
Το παρόν μήνυμα περιέχει εμπιστευτικές πληροφορίες και προορίζεται αποκλειστικά για τον αναφερόμενο παραλήπτη. Αν το λάβατε από λάθος:
Μην το κοινοποιήσετε, αναπαράγετε ή διανείμετε.
Ενημερώστε τη διεύθυνση  :  support@uhc.gr.
Διαγράψτε το μήνυμα από το σύστημά σας.
Σημείωση: Οι επικοινωνίες μέσω διαδικτύου ενδέχεται να μην είναι ασφαλείς. Η Κεντρική Υπηρεσία ΓΕΜΗ δεν φέρει νομική ευθύνη για τυχόν βλάβες από κακόβουλο λογισμικό. Οι απόψεις που εκφράζονται ανήκουν αποκλειστικά στον αποστολέα και δεν αντιπροσωπεύουν απαραίτητα την Υπηρεσία.
Με εκτίμηση,
Η Υπηρεσία Υποστήριξης & Ανάπτυξης ΠΣ ΓΕΜΗ και ΥΜΣ
"""

# Step 2: Make the messages list

messages = [   # fill this in
    {"role": "system"; "content": system_prompt},
    {"role": "user"; "content": user_prompt}
]
# Step 3: Call OpenAI

response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages)

# Step 4: print the result

print(response.choices[0].message.content)

## An extra exercise for those who enjoy web scraping

You may notice that if you try `display_summary("https://openai.com")` - it doesn't work! That's because OpenAI has a fancy website that uses Javascript. There are many ways around this that some of you might be familiar with. For example, Selenium is a hugely popular framework that runs a browser behind the scenes, renders the page, and allows you to query it. If you have experience with Selenium, Playwright or similar, then feel free to improve the Website class to use them. In the community-contributions folder, you'll find an example Selenium solution from a student (thank you!)

# Sharing your code

I'd love it if you share your code afterwards so I can share it with others! You'll notice that some students have already made changes (including a Selenium implementation) which you will find in the community-contributions folder. If you'd like add your changes to that folder, submit a Pull Request with your new versions in that folder and I'll merge your changes.

If you're not an expert with git (and I am not!) then GPT has given some nice instructions on how to submit a Pull Request. It's a bit of an involved process, but once you've done it once it's pretty clear. As a pro-tip: it's best if you clear the outputs of your Jupyter notebooks (Edit >> Clean outputs of all cells, and then Save) for clean notebooks.

Here are good instructions courtesy of an AI friend:  
https://chatgpt.com/share/677a9cb5-c64c-8012-99e0-e06e88afd293